In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

%matplotlib inline
plt.rcParams['figure.figsize'] = (8.0, 6.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

Exercise 1

I will choose face from George W Bush, Gerhard Schroeder and Tony Blair. These are the three classes. Their target value will be 1,2,3 respectively. I will use 20 percents of the data as test dataset.

In [28]:
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split

P = 7
lfw = fetch_lfw_people(min_faces_per_person=70)
mask = np.isin(lfw.target, [3, 4, 6])
X= lfw.images[mask]
y= lfw.target[mask]
y[y==3] = 0
y[y==4] = 1
y[y==6] = 2


HEIGHT = 62
WIDTH  = 47

X_stretch = []
LOC_list = []
for num,i in enumerate(X):
    H_pad = P - HEIGHT % P
    W_pad = P - WIDTH % P
    image1_padded = np.pad(i,((0,H_pad),(0, W_pad)),mode='constant')
    H_af, W_af = image1_padded.shape
    row = None
    for h in range(int(H_af/P)):
        for w in range(int(W_af/P)):
            if row is None:
                row = image1_padded[P*h:P*(h+1), P*w:P*(w+1)].reshape(1,-1)
            else:
                row = np.concatenate((row, image1_padded[P*h:P*(h+1), P*w:P*(w+1)].reshape(1,-1)))
            if num == 0:
                LOC_list.append(np.array([P*h+P//2,P*w +P//2]))
            
    X_stretch.append(row)

X_stretch = np.stack(X_stretch)

X_train, X_test, y_train, y_test = train_test_split(
    X_stretch, y, test_size=0.2, random_state=1209)

In [27]:
# X_r = np.stack(X_stretch)
# X_r.shape
len(LOC_list)

63

This function keeps track of the location $z_i$ of the patches and represents it as two-dim location, (x, y). Then we can compute the loss of it. And p I choose is 7, to decrease the computation time.

In [29]:
def k_gaussian(image1, image2, sigma= 1, sigma2= 1):
    H_af, W_af = image1_padded.shape
    result = []
    
    image_len = image1.shape[0]
    for h in range(image_len):
        for w in range(h, image_len):
            loc1 = LOC_list[h]
            loc2 = LOC_list[w]
            norm1 = np.sqrt(np.sum((image1[h])**2))
            norm2 = np.sqrt(np.sum((image2[w])**2))
            dist = np.sum((image1[h]/norm1 - image2[w]/norm2)**2)
            if h == w:
                result.append(np.exp(- dist/sigma) * np.exp(-np.linalg.norm(loc1-loc2)**2/sigma2) )
            if w > h:
                result.append(2* np.exp(- dist/sigma) * np.exp(-np.linalg.norm(loc1-loc2)**2/sigma2))
    return np.sum(result)

#try to compute between the first and second in train_data
k_gaussian(X_train[0],X_train[1])

56.554456779986452

In [ ]:
def nearest_mean(X_train, X_test, y_train, y_test,sigma, sigma2):
    n_train, n_test = len(X_train),len(X_test)
    result = np.zeros((n_train, n_test))
    for i in range(n_train):
        for j in range(n_test):
            result[i,j]= k_gaussian(X_train[i], X_test[j], sigma=sigma, sigma2 = sigma2)
    mask1 = (y_train == 0).astype(np.int)
    mask2 = (y_train == 1).astype(np.int)
    mask3 = (y_train == 2).astype(np.int)
    value_1 = np.dot(result.T, mask1[:, np.newaxis])/np.sum(mask1)
    value_2 = np.dot(result.T, mask2[:, np.newaxis])/np.sum(mask2)
    value_3 = np.dot(result.T, mask3[:, np.newaxis])/np.sum(mask3)
    
    value = np.concatenate((value_1,value_2,value3),axis=1)
    prediction = np.argmax(value, axis=1)
    error_rate =np.mean(prediction == y_test)
    return error_rate


guassian_result = []
dist_result = []
parameter_list1 = [0.001,0.01,0.1,1,10,100]
parameter_list2 = [0.01, 1, 10]
for sigma2 in parameter_list2:
    for sigma in parameter_list1:
        error_rate = nearest_mean(X_train, X_test, y_train, y_test,sigma=sigma, sigma2=sigma2)
        guassian_result.append(error_rate)
        print('Finish one round!')